<a href="https://colab.research.google.com/github/jagatabhay/TSAI/blob/master/S11/S11_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import time

In [2]:
!git clone https://github.com/jagatabhay/TSAI.git

Cloning into 'TSAI'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 408 (delta 32), reused 0 (delta 0), pack-reused 355
Receiving objects: 100% (408/408), 2.44 MiB | 4.72 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [3]:
from TSAI.S11 import CustomResNet
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device : ",device)
model = CustomResNet.CustomResNet().to(device)
print("Model Loaded Successfully ")

Device :  cuda
Model Loaded Successfully 


In [4]:
from TSAI.S11 import albumentationstransform
albumentationstransform_train_transforms = albumentationstransform.train_transforms()
from TSAI.S11 import dataloader
trainloader , testloader = dataloader.datasetloader(albumentationstransform_train_transforms , batchsize = 512 , numwork = 4 )
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

REQUIRED LIBRARIES LOADED...


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
INFO : Trainloader and Testloader Done


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0000001 , momentum=0.9 ,weight_decay = 0.01)

In [ ]:
from TSAI.S11 import LRScheduler
lrfinder = LRScheduler.LRFinder(model, optimizer , criterion , device = "cuda")
lrfinder.range_test(trainloader , end_lr =  0.1 , num_iter = 500 , step_mode = "linear")
lrfinder.plot()

In [ ]:
lrfinder.reset()

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = next(iter(trainloader))
images, labels = dataiter

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [5]:
from TSAI.S11 import traindataset

In [6]:
print("Devive : ", device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.003 , momentum = 0.95 , weight_decay = 0.01 )
scheduler = torch.optim.lr_scheduler.OneCycleLR( optimizer , max_lr = 0.01 , div_factor = 10 , pct_start  = 5/24 ,
                                                 cycle_momentum = True , epochs = 24 , steps_per_epoch = len(trainloader),
                                                 final_div_factor = 1 , anneal_strategy = 'linear' )

start_time = time.time()
traindataset.traindataset(range_ = 24 , model = model , device = device , 
                          trainloader = trainloader , optimizer = optimizer  , criterion_ = criterion ,
                          batchsize = 512 , scheduler_ = scheduler )
end_time = time.time()
print("Execution Time : ",end_time-start_time)

Devive :  cuda


/content/TSAI/S11/CustomResNet.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[1,    90] loss: 1.892
[2,    90] loss: 1.272
[3,    90] loss: 0.991
[4,    90] loss: 0.835
[5,    90] loss: 0.726
[6,    90] loss: 0.646
[7,    90] loss: 0.559
[8,    90] loss: 0.508
[9,    90] loss: 0.473
[10,    90] loss: 0.426
[11,    90] loss: 0.393
[12,    90] loss: 0.372
[13,    90] loss: 0.348
[14,    90] loss: 0.325
[15,    90] loss: 0.304
[16,    90] loss: 0.287
[17,    90] loss: 0.266
[18,    90] loss: 0.252
[19,    90] loss: 0.234
[20,    90] loss: 0.220
[21,    90] loss: 0.202
[22,    90] loss: 0.185
[23,    90] loss: 0.167
[24,    90] loss: 0.146
INFO : Finished Training of Dataset 
Execution Time :  581.6825618743896


In [7]:
from TSAI.S11 import testdataset
testdataset.testdataset(model = model , device = device , testloader = testloader )

/content/TSAI/S11/CustomResNet.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Accuracy of the Network on the 10000 test images: 91 %
